In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
df=pd.read_csv("https://raw.githubusercontent.com/krishnaik06/FSDSRegression/main/notebooks/data/gemstone.csv")
df=df.head(100)

In [151]:
df=df.drop(df.iloc[:,:1],axis=1)

In [152]:
## Independent and dependent features
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [153]:
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [154]:
x.head(2)

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05


In [155]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [156]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
#pipelines
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [157]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [158]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [159]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [160]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [162]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.312556,-0.900490,-0.041776,-0.097674,-0.071571,-0.155824,-1.056720,-0.939139,1.147417
1,-0.046918,0.713340,-1.796351,0.116570,0.134833,0.185041,0.815949,1.518420,0.466746
2,-0.988725,-0.141041,-0.626634,-1.057116,-1.084829,-1.059856,0.815949,0.289641,1.828088
3,-0.964576,-0.520765,-0.041776,-1.047801,-1.066065,-1.074677,0.815949,-0.939139,-0.894596
4,-0.843831,-0.615696,1.712800,-0.842871,-0.869042,-0.896834,-0.120386,-0.324749,-0.894596


In [163]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [167]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [168]:
regression.coef_

array([[ 6701.47208421,  -195.40126598,    44.31576028, -2990.21254431,
        -2233.73171096,  2302.59653923,   318.034346  ,  -539.53300813,
          884.15190907]])

In [169]:
regression.intercept_

array([3653.4])

In [172]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [173]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 964.969814440157
MAE: 796.6016953608591
R2 score 95.63772827691116


Lasso
Model Training Performance
RMSE: 955.3497036555029
MAE: 782.2837575572611
R2 score 95.72427264814347


Ridge
Model Training Performance
RMSE: 1069.8816688024283
MAE: 867.093070468188
R2 score 94.63763032465732


Elasticnet
Model Training Performance
RMSE: 1689.9184529343984
MAE: 1156.0092909069351
R2 score 86.62121255407867


